In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import ipywidgets as widgets
from ipywidgets import interact

In [3]:
import plotly.express as px

In [4]:
df = pd.read_csv('./data/rent_airbnb_all_features.csv')

## First, simplify columns

In [5]:
def sum_columns_and_drop(df, columns, new_column, position):
    # convert the list of column names into a numeric range
    cols_range = []
    for col in columns:
        index_no = df.columns.get_loc(col)
        cols_range.append(index_no)
    cols_range = np.array(cols_range)
    
    # insert the new colum with the sum of all given columns
    df.insert(position, new_column, df.iloc[:, cols_range].sum(axis=1))
    # drop the old columns and return the dataframe
    return df.drop(columns=columns)


In [6]:
df = sum_columns_and_drop(df, ['2 Spanish', '3 Spanish', '4 espanyols o més'], 'more than 2 Spanish', 13)
df = sum_columns_and_drop(df, ['2 foreigners', '3 foreigners', 'more than 4 foreigners'], 'more than 2 foreigners', 13)
df = sum_columns_and_drop(df, ['Up to 6 months', 'From 6 months to 12 motnhs'], 'short-term unemployment', 9)
df = df.rename(columns={'Over 12 months':'long-term unemployment'})
df = sum_columns_and_drop(df, ['man_without education', 'woman_without education', 'woman_unknown', 'man_unknown'], 'no education', 14)
df = sum_columns_and_drop(df, ['man_elementary_school', 'man_junior_high_school', 'man_senior_high_school',
                               'woman_elementary_school', 'woman_junior_high_school', 'woman_senior_high_school'], 'medium education', 15)
df = sum_columns_and_drop(df, ['man_bachelors_degree', 'woman_bachelors_degree'], 'bachelors degree', 16)


In [7]:
#groupby_neighbourhood_object.get_group('el Raval')
neighbourhoods = df['neighbourhood_name'].unique().tolist()
neighbourhoods

['Sant Pere, Santa Caterina i la Ribera',
 'el Barri Gòtic',
 'el Raval',
 'la Barceloneta',
 'Sant Antoni',
 'el Fort Pienc',
 "l'Antiga Esquerra de l'Eixample",
 "la Dreta de l'Eixample",
 "la Nova Esquerra de l'Eixample",
 'la Sagrada Família',
 'Vallcarca i els Penitents',
 "el Camp d'en Grassot i Gràcia Nova",
 'el Coll',
 'la Salut',
 'la Vila de Gràcia',
 'Can Baró',
 'Horta',
 'Montbau',
 'Sant Genís dels Agudells',
 'el Baix Guinardó',
 'el Carmel',
 'el Guinardó',
 'la Clota',
 "la Font d'en Fargues",
 'la Teixonera',
 "la Vall d'Hebron",
 'Pedralbes',
 'la Maternitat i Sant Ramon',
 'les Corts',
 'Can Peguera',
 'Canyelles',
 'Ciutat Meridiana',
 'Porta',
 'Torre Baró',
 'Vallbona',
 'Verdun',
 'Vilapicina i la Torre Llobeta',
 'el Turó de la Peira',
 'la Guineueta',
 'la Prosperitat',
 'la Trinitat Nova',
 'les Roquetes',
 'Baró de Viver',
 'Navas',
 'Sant Andreu',
 'el Bon Pastor',
 'el Congrés i els Indians',
 'la Sagrera',
 'la Trinitat Vella',
 'Diagonal Mar i el Front 

In [8]:
districts = df['district_name'].unique().tolist()
districts

['Ciutat Vella',
 'Eixample',
 'Gràcia',
 'Horta-Guinardó',
 'Les Corts',
 'Nou Barris',
 'Sant Andreu',
 'Sant Martí',
 'Sants-Montjuïc',
 'Sarrià-Sant Gervasi']

In [9]:
variables = df.columns.drop(['year', 'quarter', 'district_code', 'district_name', 'neighbourhood_code', 'neighbourhood_name']).to_list()
variables

['price_month(€)',
 'price_m2_month(€)',
 'short-term unemployment',
 'long-term unemployment',
 '1 Spanish',
 '1 foreigner',
 'more than 2 foreigners',
 'more than 2 Spanish',
 'no education',
 'medium education',
 'bachelors degree',
 'airbnb_price(€)',
 'n_airbnb_listings',
 'availability_365',
 'airbnb_ratio_business',
 'airbnb_professional_ratio',
 'airbnb_mega_host_ratio']

In [10]:
df['quarter'] = df['quarter'].apply(pd.to_datetime).values

In [11]:
@interact
def my_plot(distr = districts, 
            var=variables):
    
#     fig = px.scatter(df[df['district_name'] == distr], x='quarter', y=var, color='neighbourhood_name',
#                         template='simple_white')
    fig = px.line(df[df['district_name'] == distr], x='quarter', y=var, color='neighbourhood_name',
                  template='simple_white')
    fig.show()


interactive(children=(Dropdown(description='distr', options=('Ciutat Vella', 'Eixample', 'Gràcia', 'Horta-Guin…